Data Understanding

In [2]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns
import tldextract
from sklearn.preprocessing import StandardScaler

Import Data

In [3]:
# Load datasets
rating_path = '../data/rating.csv'
domains_location_path = '../data/domains_location.csv'
traffic_path = '../data/traffic.csv'

rating_df = pd.read_csv(rating_path)
domains_location_df = pd.read_csv(domains_location_path)
traffic_data_df = pd.read_csv(traffic_path)

FileNotFoundError: [Errno 2] No such file or directory: '../data/rating.csv'

In [ ]:
# Display first few rows of each dataset
print("Rating.csv:\n", tabulate(rating_df.head(), headers='keys', tablefmt='psql'))
print("\nDomains Location.csv:\n", tabulate(domains_location_df.head(), headers='keys', tablefmt='psql'))
print("\nTraffic Data.csv:\n", tabulate(traffic_data_df.head(), headers='keys', tablefmt='psql'))

In [ ]:
# Function to print column names and shape
def print_info(df, df_name):
    print(f"{df_name} columns: {df.columns.tolist()}")
    print(f"{df_name} shape: The df has {df.shape[0]} rows and {df.shape[1]} columns.\n")

# Print information for each data set
print_info(rating_df, 'rating_df')
print_info(domains_location_df, 'domains_location_df')
print_info(traffic_data_df, 'traffic_data_df')

Feature Engineering

In [ ]:
# Function to extract domain
def extract_domain(url):
    ext = tldextract.extract(url)
    return f"{ext.domain}.{ext.suffix}"

# Apply the function to the 'url' column
rating_df['domain'] = rating_df['url'].apply(extract_domain)

# Display the first few rows to verify
print(rating_df[['url', 'domain']].head())


In [ ]:
# Get unique counts of domains
unique_rating_domains = rating_df['domain'].nunique()
unique_location_domains = domains_location_df['SourceCommonName'].nunique()
unique_traffic_domains = traffic_data_df['Domain'].nunique()

print(f"Unique domains in rating_df: {unique_rating_domains}")
print(f"Unique domains in domains_location_df: {unique_location_domains}")
print(f"Unique domains in traffic_data_df: {unique_traffic_domains}")

In [ ]:
# Define the mapping for sentiment values
sentiment_mapping = {'Positive': 1, 'Neutral': 0, 'Negative': -1}

# Apply the mapping to the title_sentiment column
rating_df['title_sentiment_score'] = rating_df['title_sentiment'].map(sentiment_mapping)

# Verify the changes
print(rating_df[['title_sentiment', 'title_sentiment_score']].head())


In [ ]:
# Merge rating_df with domains_location_df
merged_df = pd.merge(rating_df, domains_location_df, left_on='domain', right_on='SourceCommonName', how='left')

# Merge the resulting DataFrame with traffic_data_df
final_df = pd.merge(merged_df, traffic_data_df, left_on='domain', right_on='Domain', how='left')

# Display the first few rows to verify
print(final_df.head())

In [ ]:
# Function to print column names and shape
def print_info(df, df_name):
    print(f"{df_name} columns: {df.columns.tolist()}")
    print(f"{df_name} shape: The df has {df.shape[0]} rows and {df.shape[1]} columns.\n")

# Print information for the data set
print_info(final_df, 'final_df')

Data Preparation

In [ ]:
# Function to check for missing values
def check_missing_values(df):
    total_rows = len(df)
    missing_info = df.isnull().sum()
    for column, missing_count in missing_info.items():
        if missing_count > 0:
            missing_percentage = (missing_count / total_rows) * 100
            print(f"The {column} column has {missing_count} missing values which is {missing_percentage:.2f}% of the column.")

check_missing_values(final_df)

Handle Missing Values

In [ ]:
# Drop source_id column
if 'source_id' in final_df.columns:
    final_df = final_df.drop(columns=['source_id'])

# Impute missing values
fill_values = {
    'author': 'Unknown',
    'description': 'No description available',
    'url_to_image': 'No image available',
    'category': 'Uncategorized',
    'SourceCommonName': 'Unknown',
    'location': 'Unknown',
    'Country': 'Unknown'
}

final_df.fillna(value=fill_values, inplace=True)

# Verify that there are no missing values
check_missing_values(final_df)

In [ ]:
# Summary statistics and data types
print("\nfinal_df summary:")
print(final_df.describe(include='all'))
print("\nfinal_df data types:")
print(final_df.dtypes)

Dropping Irrelevant Columns and Feature Engineering

In [ ]:
# Convert 'published_at' to datetime with explicit format
final_df['published_at'] = pd.to_datetime(final_df['published_at'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')

# Extract date-related features
final_df['published_year'] = final_df['published_at'].dt.year
final_df['published_month'] = final_df['published_at'].dt.month
final_df['published_day'] = final_df['published_at'].dt.day

# Check for NaT values in the 'published_at' column
print(final_df['published_at'].isna().sum())


In [ ]:
# Check for missing values
print(check_missing_values(final_df))

In [ ]:
columns_to_drop = [
    #'article_id', 'source_name', 'url_to_image', 'article', 
    #'TLD', 'IDN_Domain', 'IDN_TLD', 'PrevGlobalRank', 
    #'PrevTldRank', 'PrevRefSubNets', 'PrevRefIPs', 
    'published_at',
    'published_year', 'published_month', 'published_day'
]

columns_to_drop = [col for col in columns_to_drop if col in final_df.columns]

final_df = final_df.drop(columns=columns_to_drop)

print(check_missing_values(final_df))

# Verify the changes
print(final_df.columns)

In [ ]:
# Calculate content length
final_df['content_length'] = final_df['content'].str.len()

# Calculate title word count
final_df['title_word_count'] = final_df['title'].str.split().apply(len)

# Convert title_sentiment to numerical scores if needed
sentiment_mapping = {'positive': 1, 'neutral': 0, 'negative': -1}
final_df['title_sentiment_score'] = final_df['title_sentiment'].map(sentiment_mapping)

# Display the first few rows to verify
print(final_df.head())

Outliers

In [ ]:
# Function to plot box plots for numerical columns with enhanced visuals
def plot_boxplots(df):
    # Select numerical columns
    numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
    
    # Set the overall style
    sns.set(style="whitegrid")

    for column in numerical_columns:
        # Check if the column has more than one unique value and no NaN values
        if df[column].nunique() > 1 and df[column].notna().sum() > 0:
            plt.figure(figsize=(12, 6))  # Increase figure size for better readability
            sns.boxplot(x=df[column], color='skyblue')  # Add a color for distinction
            plt.title(f'Box Plot of {column}', fontsize=16)  # Add a title with larger font size
            plt.xlabel(column, fontsize=14)  # Label the x-axis with a larger font size
            plt.grid(True, linestyle='--', alpha=0.7)  # Add gridlines for better readability
            plt.show()
        else:
            print(f"Skipping {column} due to insufficient data for plotting.")

# Call the function to plot boxplots for final_df
plot_boxplots(final_df)

In [ ]:
# Function to identify outliers using Z-score
def find_outliers_zscore(df, threshold=3):
    # Select numerical columns
    numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
    
    outliers_dict = {}  # Dictionary to store outliers by column
    
    for column in numerical_columns:
        # Calculate Z-scores
        z_scores = np.abs((df[column] - df[column].mean()) / df[column].std())
        
        # Identify outliers
        outliers = df[column][z_scores > threshold]
        
        # Store outliers in the dictionary
        if not outliers.empty:
            outliers_dict[column] = outliers
    
    return outliers_dict

# Call the function to find outliers for final_df
outliers = find_outliers_zscore(final_df)

# Print the outliers found
if outliers:
    for column, outlier_values in outliers.items():
        print(f"Outliers detected in column '{column}':")
        print(outlier_values)
        print("\n")
else:
    print("No outliers detected using the Z-score method.")


In [ ]:
# Function to plot histograms and display descriptive statistics
def plot_histograms(df):
    numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
    for column in numerical_columns:
        plt.figure(figsize=(10, 5))
        sns.histplot(df[column], kde=True, color='blue')
        plt.title(f'Distribution of {column}')
        plt.show()

        # Descriptive statistics
        mean = df[column].mean()
        median = df[column].median()
        std_dev = df[column].std()
        print(f"{column} - Mean: {mean}, Median: {median}, Std Dev: {std_dev}\n")

plot_histograms(final_df)


In [ ]:
# Function to plot bar charts for categorical data and display value counts
#def plot_bar_charts(df):
    #categorical_columns = df.select_dtypes(include=['object']).columns
    #for column in categorical_columns:
        #plt.figure(figsize=(10, 5))
        #df[column].value_counts().plot(kind='bar', color='green')
        #plt.title(f'Bar Chart of {column}')
        #plt.show()

        # Display value counts
        #print(f"Value Counts for {column}:")
        #print(df[column].value_counts(), "\n")

#plot_bar_charts(final_df)


In [ ]:
# Group sources with less than a threshold into 'Other'
threshold = 500
source_counts = final_df['source_name'].value_counts()
grouped_sources = source_counts[source_counts >= threshold]
grouped_sources['Other'] = source_counts[source_counts < threshold].sum()

# Plot the grouped sources as a bar chart
plt.figure(figsize=(12, 6))
grouped_sources.plot(kind='bar', color='blue')
plt.title('Article Count by Source (Grouped)')
plt.xlabel('Source Name')
plt.ylabel('Count')
plt.show()

# Plot the same data as a pie chart
plt.figure(figsize=(10, 10))
grouped_sources.plot(kind='pie', autopct='%1.1f%%', startangle=140)
plt.title('Article Distribution by Source (Grouped)')
plt.ylabel('')  # Hide the y-label for a cleaner pie chart
plt.show()


In [ ]:
# Calculate the percentage contribution of each source
percentage_contribution = (source_counts / len(final_df)) * 100
print(percentage_contribution)

Exploratory Data Analysis

1.1 Top and Bottom 10 Websites by News Articles Count

In [ ]:
# Top 10 websites with the largest count of news articles
top_10_websites = final_df['source_name'].value_counts().head(10)
print("Top 10 Websites by Article Count:\n", top_10_websites)

# Bottom 10 websites with the smallest count of news articles
bottom_10_websites = final_df['source_name'].value_counts().tail(10)
print("\nBottom 10 Websites by Article Count:\n", bottom_10_websites)


1.2 Top and Bottom 10 Websites by Visitor Traffic

In [ ]:
# Define estimated visitors
def estimate_visitors(row):
    # Extract relevant columns
    global_rank = row['GlobalRank']
    ref_subnets = row['RefSubNets']
    ref_ips = row['RefIPs']
    
    # Handle potential division by zero or missing values
    if global_rank <= 0:  # GlobalRank should be a positive integer
        return None
    if pd.isna(ref_subnets) or pd.isna(ref_ips):
        return None

    # Example scaling factor
    scaling_factor = 10000
    
    # Calculate estimated visitors
    estimated_visitors = (1 / global_rank) * (ref_subnets + ref_ips) * scaling_factor
    return estimated_visitors

# Apply the function to the DataFrame
final_df['EstimatedVisitors'] = final_df.apply(estimate_visitors, axis=1)

# Display the first few rows of the updated DataFrame
print(final_df[['Domain', 'GlobalRank', 'RefSubNets', 'RefIPs', 'EstimatedVisitors']].head())


In [ ]:
# Top 10 websites by visitor traffic
top_10_traffic_websites = final_df[['source_name', 'EstimatedVisitors']].groupby('source_name').mean().sort_values(by='EstimatedVisitors', ascending=False).head(10)
print("Top 10 Websites by Visitor Traffic:\n", top_10_traffic_websites)

# Bottom 10 websites by visitor traffic
bottom_10_traffic_websites = final_df[['source_name', 'EstimatedVisitors']].groupby('source_name').mean().sort_values(by='EstimatedVisitors').head(10)
print("\nBottom 10 Websites by Visitor Traffic:\n", bottom_10_traffic_websites)


1.3 Countries with the Highest Number of News Media Organizations

In [ ]:
# Count of news media organizations (domains) by country
country_media_count = final_df['location'].value_counts()
top_countries_media = country_media_count.head(10)
print("Countries with the Highest Number of News Media Organizations:\n", top_countries_media)


1.4 Countries with Many Articles Written About Them

In [ ]:
# Grouping by location to find out the number of articles related to each country
articles_about_countries = final_df['location'].value_counts()
print("Countries with Many Articles Written About Them:\n", articles_about_countries.head(10))


1.5 Websites Reporting About Specific Regions

In [ ]:
# Extract unique country names or codes
unique_countries = final_df['Country'].dropna().unique()

# Convert to a DataFrame for better visualization (optional)
unique_countries_df = pd.DataFrame(unique_countries, columns=['Country'])

# Display the unique country names or codes
print(unique_countries_df)

In [ ]:
# Updated regions dictionary with new countries
regions = {
    'North America': ['United States'],
    'Europe': ['United Kingdom', 'Russia'],  # Including Russia here for a broader European context
    'Asia': ['India'],
    'Africa': ['Nigeria'],
    'Middle East': ['Saudi Arabia', 'UAE', 'Iran', 'Israel', 'Turkey'],
    'South America': ['Brazil', 'Argentina', 'Chile'],
    'Oceania': ['Australia', 'New Zealand'],
    'Other': []  # To include any additional or unspecified regions
}

# Function to categorize a website based on its location
def categorize_region(location):
    for region, countries in regions.items():
        if location in countries:
            return region
    return 'Other'

# Apply the function to categorize each entry
final_df['region'] = final_df['Country'].apply(categorize_region)

# Count of websites reporting about specific regions
region_report_counts = final_df['region'].value_counts()
print("Websites Reporting About Specific Regions:\n", region_report_counts)


1.6 Websites with Highest Count of Positive, Neutral, and Negative Sentiment

In [ ]:
# Assuming sentiment column exists with values 'positive', 'neutral', 'negative'
sentiment_summary = final_df.groupby('source_name')['title_sentiment'].value_counts().unstack().fillna(0)

# Descriptive statistics for sentiment
sentiment_stats = sentiment_summary.describe()
print("Descriptive Statistics for Sentiments:\n", sentiment_stats)


1.7 Compare Impact of Mean and Median Sentiment

In [ ]:
# Group by 'source_name' and calculate sentiment counts
sentiment_counts = final_df.groupby('source_name')['title_sentiment'].value_counts().unstack().fillna(0)

# Rename columns for clarity
sentiment_counts.columns = ['negative_count', 'neutral_count', 'positive_count']

# Get the top websites for each sentiment type
top_positive_websites = sentiment_counts[['positive_count']].nlargest(10, 'positive_count')
top_neutral_websites = sentiment_counts[['neutral_count']].nlargest(10, 'neutral_count')
top_negative_websites = sentiment_counts[['negative_count']].nlargest(10, 'negative_count')

print("Top Websites by Positive Sentiment Count:\n", top_positive_websites)
print("\nTop Websites by Neutral Sentiment Count:\n", top_neutral_websites)
print("\nTop Websites by Negative Sentiment Count:\n", top_negative_websites)


In [ ]:
# Select top 10 domains by visitor traffic
top_10_domains = final_df.groupby('source_name')['EstimatedVisitors'].sum().nlargest(10).index

# Filter data for top 10 domains
top_10_df = final_df[final_df['source_name'].isin(top_10_domains)]

# Calculate sentiment counts for top 10 domains
sentiment_counts_top_10 = top_10_df.groupby('source_name')['title_sentiment'].value_counts().unstack().fillna(0)

# Calculate mean and median for each sentiment
sentiment_summary = sentiment_counts_top_10.agg(['mean', 'median', 'var'])

print("Sentiment Summary for Top 10 Domains:\n", sentiment_summary)

# Plot comparison of mean and median
# Data for plotting
sentiment_types = ['Negative', 'Neutral', 'Positive']
mean_values = [455.4, 1613.7, 334.3]
median_values = [247.5, 1046.0, 112.0]

# Create a bar plot for mean and median sentiment counts
x = range(len(sentiment_types))  # Position of bars

plt.figure(figsize=(12, 6))
plt.bar(x, mean_values, width=0.4, label='Mean', color='blue', align='center')
plt.bar([p + 0.4 for p in x], median_values, width=0.4, label='Median', color='orange', align='center')

plt.xlabel('Sentiment Type')
plt.ylabel('Count')
plt.title('Comparison of Mean and Median Sentiment Counts for Top 10 Domains')
plt.xticks([p + 0.2 for p in x], sentiment_types)
plt.legend()
plt.show()

1.8 Distribution of Sentiments for Top 10 Domains

In [ ]:
# Get the top websites for each sentiment type
top_positive_websites = sentiment_counts[['positive_count']].nlargest(10, 'positive_count').index
top_neutral_websites = sentiment_counts[['neutral_count']].nlargest(10, 'neutral_count').index
top_negative_websites = sentiment_counts[['negative_count']].nlargest(10, 'negative_count').index

# Combine all top domains into a single list
top_10_domains = set(top_positive_websites) | set(top_neutral_websites) | set(top_negative_websites)

# Filter data for top 10 domains
top_10_sentiments = final_df[final_df['source_name'].isin(top_10_domains)]

# Ensure that 'title_sentiment' is used correctly (assuming it's categorical)
if 'title_sentiment' not in top_10_sentiments.columns:
    raise ValueError("Column 'title_sentiment' not found in the data.")

# Plot sentiment distribution
plt.figure(figsize=(14, 8))
sns.histplot(data=top_10_sentiments, x='title_sentiment', hue='source_name', multiple='stack', bins=30, palette='tab10')

plt.title('Sentiment Distribution for Top 10 Domains by Sentiment Counts')
plt.xlabel('Sentiment')
plt.ylabel('Frequency')
plt.legend(title='Source Name', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

2. Compare Content Metadata Across Sites
2.1 Distribution of Raw Message Lengths Across Sites

In [ ]:
# Distribution of message lengths by site
plt.figure(figsize=(14, 7))
sns.boxplot(data=final_df, x='source_name', y='content_length')
plt.xticks(rotation=90)
plt.title('Distribution of Raw Message Lengths Across Sites')
plt.show()


2.2 Distribution of Title Word Counts Across Sites

In [ ]:
# Distribution of title word counts by site
plt.figure(figsize=(14, 7))
sns.boxplot(data=final_df, x='source_name', y='title_word_count')
plt.xticks(rotation=90)
plt.title('Distribution of Title Word Counts Across Sites')
plt.show()


3. Impact of Frequent Reporting and Sentiment on Global Ranking

In [ ]:
# Convert title_sentiment to numeric, forcing errors to NaN
final_df['title_sentiment'] = pd.to_numeric(final_df['title_sentiment'], errors='coerce')

# Remove rows with NaN in title_sentiment if any
final_df = final_df.dropna(subset=['title_sentiment'])

In [ ]:
# 1. Calculate total number of articles and average sentiment score for each website
reporting_sentiment_df = final_df.groupby('source_name').agg(
    total_articles=('article_id', 'count'),
    average_sentiment=('title_sentiment', 'mean')
).reset_index()

# 2. Perform correlation analysis
correlation = reporting_sentiment_df['total_articles'].corr(reporting_sentiment_df['average_sentiment'])
print(f"Correlation between total articles and average sentiment score: {correlation:.2f}")

# 3. Visualize the relationship using a scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x='total_articles', y='average_sentiment', data=reporting_sentiment_df)
plt.title('Impact of Reporting Frequency on Sentiment Score')
plt.xlabel('Total Number of Articles')
plt.ylabel('Average Sentiment Score')
plt.show()